In [2]:
import requests
import re
from bs4 import BeautifulSoup

In [3]:
url = "https://www.transfermarkt.it/erling-haaland/profil/spieler/418560"
player_id = url.split('/')[-1]

In [4]:
headers = {
    'accept': '*/*',
    'accept-language': 'it-IT,it;q=0.9,en-US;q=0.8,en;q=0.7',
    'cache-control': 'max-age=0',
    'priority': 'u=1, i',
    'referer': 'https://www.sofascore.com/it/torneo/calcio/europe/uefa-champions-league/7',
    'sec-ch-ua': '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
    'x-requested-with': '3447eb',
}

In [5]:
response = requests.get(url, headers=headers)
response.status_code

200

In [7]:
soup = BeautifulSoup(response.content, 'html.parser')

In [8]:
player_name = soup.select_one('h1[class="data-header__headline-wrapper"]').text.split('\n')[-1].strip()
player_number = soup.select_one('span[class="data-header__shirt-number"]').text.strip().replace('#', '')

In [9]:
player_name

'Erling Haaland'

In [10]:

# Let's use regex to find the player's contract expiry date
player_contract_expiry = re.search("Contract expires: (.*)", soup.text).group(1)

# Now let's add some more complex regex patterns to find the player's birthplace
player_birthplace = re.search("Place of birth:.*?([A-z].*?) ", soup.text, re.DOTALL).group(1).strip()

# We can reuse both of those ideas to find other player information but just replacing the name

# We might have to switch a couple of things though in the regex pattern like accounting for two words
player_agent = re.search("Agent:.*?([A-z].*?)\n", soup.text, re.DOTALL).group(1).strip()

# We can also use regex to find the player's height which has numbers a comma and letters
player_height = re.search("Height:.*?([0-9].*?)\n", soup.text, re.DOTALL).group(1).strip()

In [12]:
api_endpoints = [
    f"marketValueDevelopment/graph/{player_id}",
    f"transferHistory/list/{player_id}",
    f"player/{player_id}/performance"
]

for endpoint in api_endpoints:
    response = requests.get(
        'https://www.transfermarkt.us/ceapi/' + endpoint,
        headers=headers
    ).json()
    print(response)

{'list': [{'x': 1482015600000, 'y': 200000, 'mw': '€200k', 'datum_mw': 'Dec 18, 2016', 'verein': 'Bryne FK', 'age': '16', 'wappen': 'https://tmssl.akamaized.net/images/wappen/profil/1057.png?lm=1480871779'}, {'x': 1513983600000, 'y': 300000, 'mw': '€300k', 'datum_mw': 'Dec 23, 2017', 'verein': 'Molde FK', 'age': '17', 'wappen': 'https://tmssl.akamaized.net/images/wappen/profil/687.png?lm=1409159512'}, {'x': 1536530400000, 'y': 2000000, 'mw': '€2.00m', 'datum_mw': 'Sep 10, 2018', 'verein': 'Molde FK', 'age': '18', 'wappen': ''}, {'x': 1546124400000, 'y': 5000000, 'mw': '€5.00m', 'datum_mw': 'Dec 30, 2018', 'verein': 'Molde FK', 'age': '18', 'wappen': ''}, {'x': 1559512800000, 'y': 5000000, 'mw': '€5.00m', 'datum_mw': 'Jun 3, 2019', 'verein': 'Red Bull Salzburg', 'age': '18', 'wappen': 'https://tmssl.akamaized.net/images/wappen/profil/409_1557781653.png?lm=1557781653'}, {'x': 1567461600000, 'y': 12000000, 'mw': '€12.00m', 'datum_mw': 'Sep 3, 2019', 'verein': 'Red Bull Salzburg', 'age': '

: 